In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

In order to deal with the complexity of managing objects, people developed a new class of systems called ORM.

#### The Old Way of Writing Database Code in Python

In [2]:
import sqlite3

conn = sqlite3.connect("example.db")

c = conn.cursor()
c.execute(
    """
          CREATE TABLE person
          (id INTEGER PRIMARY KEY ASC, name varchar(250) NOT NULL)
          """
)
c.execute(
    """
          CREATE TABLE address
          (id INTEGER PRIMARY KEY ASC, street_name varchar(250), street_number varchar(250),
           post_code varchar(250) NOT NULL, person_id INTEGER NOT NULL,
           FOREIGN KEY(person_id) REFERENCES person(id))
          """
)

c.execute(
    """
          INSERT INTO person VALUES(1, 'pythoncentral')
          """
)
c.execute(
    """
          INSERT INTO address VALUES(1, 'python road', '1', '00000', 1)
          """
)

conn.commit()
conn.close()

In [3]:
conn = sqlite3.connect("example.db")

c = conn.cursor()
c.execute("SELECT * FROM person")
print(c.fetchall())
c.execute("SELECT * FROM address")
print(c.fetchall())
conn.close()

[(1, 'pythoncentral')]
[(1, 'python road', '1', '00000', 1)]


### Python's SQLAlchemy and Declarative

There are three most important components in writing SQLAlchemy code:

* A *Table* that represents a table in a database.
* A *mapper* that maps a Python class to a table in a database.
* A class object that defines how a database record maps to a normal Python object.

In [4]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

Base = declarative_base()


class Person(Base):
    __tablename__ = "person"
    # Here we define columns for the table person
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)


class Address(Base):
    __tablename__ = "address"
    # Here we define columns for the table address.
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    street_name = Column(String(250))
    street_number = Column(String(250))
    post_code = Column(String(250), nullable=False)
    person_id = Column(Integer, ForeignKey("person.id"))
    person = relationship(Person)


# Create an engine that stores data in the local directory's
# sqlalchemy_example.db file.
engine = create_engine("sqlite:///sqlalchemy_example.db")

# Create all tables in the engine. This is equivalent to "Create Table"
# statements in raw SQL.
Base.metadata.create_all(engine)

In [5]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [6]:
engine = create_engine("sqlite:///sqlalchemy_example.db")
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()

# Insert a Person in the person table
new_person = Person(name="new person")
session.add(new_person)
session.commit()

# Insert an Address in the address table
new_address = Address(post_code="00000", person=new_person)
session.add(new_address)
session.commit()

In [7]:
# Make a query to find all Persons in the database
session.query(Person).all()

In [8]:
# Return the first Person from all Persons in the database
person = session.query(Person).first()

In [9]:
person.name

'new person'

In [10]:
# Retrieve one Address whose person field is point to the person object
address = session.query(Address).filter(Address.person == person).one()

In [11]:
address.post_code

'00000'